In [1]:
from selenium import webdriver
import time
import datetime

run_date = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

# chromeドライバーを起動する時のオプションを設定
options = webdriver.ChromeOptions()

options.add_argument("--headless")  # ヘッドレスで起動
options.add_argument('--no-sandbox') # 仮想環境下では、sandboxで起動すると失敗するので無効にする
options.add_argument('--disable-gpu') # ヘッドレスモードで起動するときに必要
options.add_argument('--window-size=1280,1024')  # 画面サイズの指定
options.add_argument('--disable-dev-shm-usage')  # メモリファイルでshm利用しない様にする
options.add_argument("--disable-application-cache")  # HTML5のApplication Cacheを無効にします。
options.add_argument('--no-sandbox') # google-chrome-stableが動くために必要
options.add_argument('--hide-scrollbars') # スクロールバーを隠す
options.add_argument('--enable-logging') # console.log　保存用
options.add_argument('--log-level=0')
options.add_argument('--v=99')
options.add_argument('--ignore-certificate-errors') # SSLセキュリティ証明書のエラーページを表示しない
options.add_argument('--incognito') # シークレット モードincognito.gifで起動

# user agent
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36' )

# chromeドライバーを起動
driver = webdriver.Chrome(options=options)

print("chromeドライバー OK")

chromeドライバー OK


In [17]:
# キーワードセット
keyword = "ワイン"

# amazonを開く
driver.get("https://www.amazon.co.jp/s?k={}".format(keyword))

# 画面が表示されるまで待つ
time.sleep(3)

# ページタイトル取得
print(driver.title)

#縦と横のサイズを取得してキャプチャを取得する   
page_width = driver.execute_script('return document.body.scrollWidth')
page_height = driver.execute_script('return document.body.scrollHeight')        
driver.set_window_size(page_width, page_height)


# スクリーンショットを撮る
driver.save_screenshot("./output/{}_{}.png".format(keyword, run_date))
print("get screenshot")

# html取得
html = driver.page_source
with open('./output/{}_{}.html'.format(keyword, run_date), 'w', encoding='utf-8') as f:
    f.write(html)
print("get html")

Amazon.co.jp : ワイン
get screenshot
get html


In [22]:
# TOP & BOTTOM 広告枠
ad_list = driver.find_elements_by_xpath("//span[@data-component-type='s-ads-metrics']")

# ページ上部の広告取得
print("===================");
print("TOP AD list");
print("===================");
top_ads = driver.find_elements_by_xpath(".//div[contains(@class, '_multi-card-creative-desktop_DesktopContainer_content__')]//div[@data-asin]")
if len(top_ads) <= 0:
    top_ads = driver.find_elements_by_xpath(".//div[@data-cel-widget='MultiCardCreativeDesktop']//div[@data-asin]")

if len(top_ads) > 0:
    for top_ad in top_ads:
        asin = top_ad.get_attribute("data-asin")

        span_elems = top_ad.find_elements_by_xpath("./div/a/span")
        if len(span_elems) <= 0:
            span_elems = top_ad.find_elements_by_xpath(".//span[@data-click-el='title']")

        product_name = span_elems[0].text if len(span_elems) >= 0 else '商品名なし'

        output_json = {asin: product_name}
        print(output_json)
            

TOP AD list
{'B07F2W1FBY': 'コノスル ビシクレタ 人気セパージュ 赤&白 6本飲みくらべセット [ 750ml×6本 ]'}
{'B001M3LP7Q': 'コノスル カベルネ・ソーヴィニヨン ビシクレ タ ・レゼルバ [ 赤ワイン ミディ…'}
{'B002IJBNX4': 'コノスル ビシクレタ ゲヴュルツトラミネール ヴァラエタル [ 白ワイン 中辛口…'}


In [23]:
print("===================");
print("BOTTOM AD list");
print("===================");
bottom_ads = driver.find_elements_by_xpath(".//div[@class='threepsl-creative' and @data-index]")

# ページ下部の広告取得
if len(bottom_ads) > 0:
    for bottom_ad in bottom_ads:
        product_name = bottom_ad.find_element_by_xpath(".//div[@data-headline]/span/span").text
        link_url = bottom_ad.find_element_by_xpath(".//div/a").get_attribute("href")
        if product_name != "":
            output_json = {product_name: link_url}
            print(output_json)

BOTTOM AD list
{'それぞれの品種の特徴が楽しめるチリワイン\u3000コノスル「お酒は20歳から」': 'https://aax-fe.amazon-adsystem.com/x/c/QrrwGw5LEb-yISXZPT7pVkQAAAF2jtqFSgcAAAH2AePtkFE/https://www.amazon.co.jp/stores/page/105CB756-8459-48BA-8A39-513D4B2BF7BB?store_ref=SB_AITTW28OKKT0D&pd_rd_w=8wG7D&pf_rd_p=00208ac8-ff11-496b-9c70-44710ac295bb&pd_rd_wg=S5x5i&pf_rd_r=SYPAKK50YNY2PJ9C24TZ&pd_rd_r=6fdb5f25-c1e0-4c1a-8428-16aad8e49539&aaxitk=DAN..T5wdPYsbWF-QIcy7Q&hsa_cr_id=9352636700703&lp_asins=B07F2W1FBY,B001M3LP7Q,B002IJBNX4&lp_mat_key=%E3%83%AF%E3%82%A4%E3%83%B3&lp_query=%E3%83%AF%E3%82%A4%E3%83%B3&lp_slot=desktop-hsa-3psl&ref_=sbx_be_s_3psl_mbd'}
{'ワイン専門店・ENOTECA（エノテカ）お酒は20歳から': 'https://aax-fe.amazon-adsystem.com/x/c/QtZgHgM7VT1s4okGXFRipmIAAAF2jtqFSgcAAAH2AUIauzI/https://www.amazon.co.jp/stores/page/9201EDFD-412B-464E-9E98-EA68D9297825?store_ref=SB_AKDD9RDOU7EHT&pd_rd_w=8wG7D&pf_rd_p=00208ac8-ff11-496b-9c70-44710ac295bb&pd_rd_wg=S5x5i&pf_rd_r=SYPAKK50YNY2PJ9C24TZ&pd_rd_r=6fdb5f25-c1e0-4c1a-8428-16aad8e49539&aaxitk=kdxa

In [9]:
# プロダクト広告取得
print("=======================")
print("GET ADHolder")
print("=======================")
asin_list = driver.find_elements_by_xpath(".//div[@data-asin and contains(@class, 'AdHolder')]")
if len(asin_list) > 0:
    for asin_item in asin_list:     
        asin = asin_item.get_attribute("data-asin")
        product_name = asin_item.find_element_by_xpath(".//h2/a/span").text
        if asin != "":
            output_json = {asin: product_name}
            print(output_json)
        

GET ADHolder
{'B075YRH3LC': 'ワインセット スパークリングワイン、赤ワイン、白ワインのミックス12本セット イタリア フランス 有名ワインセット'}
{'B07Y45XDLC': '赤ワイン セット フルボディ ALL金賞受賞 ボルドー赤ワインの飲み比べセット 12本 赤ワインセット 本場フランス赤ワイン特集'}
{'B00MXWYRL6': '全て格上AOC 格段に違う味わい フランスボルドー 赤ワイン 飲み比べ 6本セット 750ml×6本'}
{'B071HRHYJ8': '全てトリプル金賞 ボルドーワイン6本セット ソムリエ厳選 赤ワイン 750ml×6'}
{'B08FF2C372': '【ソムリエ厳選 世界の辛口 白】 シャルドネ 3本セット'}
{'B07GYNQP79': 'ルイ・ジャド ボーヌ プルミエ・クリュ オマージュ・オー・クリマ ギフト箱入 瓶 [ 2015 赤ワイン フルボディ フランス 750ml ] [ギフトBox入り]'}
{'B002JN6LGI': 'ルイ・ジャド ブルゴーニュ シャルドネ [ 2018 白ワイン 辛口 フランス 750ml ]'}
{'B08L7PCXY2': '1本あたり582円税別 厳選の旨安赤ワインだけ 赤ワイン12本セット'}
{'B07HMQLB27': '【飲めば納得 ガツンと濃厚な赤ワイン3本セット】サンタ・リタ 赤ワインセット [ 赤ワイン フルボディ チリ 750ml×3本 ]'}
{'B08H4XFXCK': '赤ワイン フルボディ セット第101弾 すべてパーカー90点以上 赤ワイン 750ml 6本セット 赤 ワインセット フ'}
{'B08FF4PX15': '【ソムリエ厳選 世界の辛口スパークリング】3本セット (スペイン・フランス・イタリア)'}
{'B08HN4DYSX': 'イタリアン アッポローニオ 濃厚赤ワインセット 6本 ワイン ワインセット'}
